# Manual Data Loading and Transformation

This notebook is for loading data from the Synthea CSV files into the PostgreSQL database one by one. This allows us to inspect, clean, and transform the data before insertion, resolving any schema mismatches.

In [2]:
%pip install psycopg2


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 3.4 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from urllib.parse import quote_plus

# Load environment variables from .env file
load_dotenv('../.env')

def get_db_engine():
    """Creates a database engine from environment variables."""
    db_user = os.getenv("DB_USER")
    raw_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    db_name = os.getenv("DB_NAME")

    if not all([db_user, raw_password, db_host, db_port, db_name]):
        raise ValueError("One or more database credentials not found. Please check your .env file.")

    # URL-encode the password to handle special characters
    db_password = quote_plus(raw_password)

    engine = create_engine(
        f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    )
    return engine

engine = get_db_engine()
print("Database engine created successfully.")

Database engine created successfully.


## 1. Load Encounters Data

In [5]:
encounters_df = pd.read_csv('../synthea_sample_data_csv_nov2021/csv/encounters.csv')
encounters_df.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,748f8357-6cc7-551d-f31a-32fa2cf84126,2019-02-17T05:07:38Z,2019-02-17T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,877.79,833.90,NaN,NaN
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,2019-03-24T05:07:38Z,2019-03-24T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,269.68,256.20,NaN,NaN
2,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,2019-05-26T05:07:38Z,2019-05-26T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,1292.75,1228.11,NaN,NaN
3,6e93bcf9-45a4-8528-0120-1c1eaa930faf,2019-07-28T05:07:38Z,2019-07-28T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,1323.87,1257.68,NaN,NaN
4,8b6787c3-4316-a0cb-899d-4746525c319f,2019-10-27T05:07:38Z,2019-10-27T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,831.76,790.17,NaN,NaN


In [6]:
# Let's look at the columns in our DataFrame
print("Columns from encounters.csv:")
print(encounters_df.columns.tolist())

Columns from encounters.csv:
['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE', 'REASONDESCRIPTION']


### Compare with the Database Schema

The `encounters` table in `db/schema.sql` has the following columns:
```sql
Id, START, STOP, PATIENT, ORGANIZATION, PROVIDER, PAYER, ENCOUNTERCLASS, CODE, DESCRIPTION, COST, REASONCODE, REASONDESCRIPTION
```

The error message told us the script tried to insert a column named `BASE_ENCOUNTER_COST`. Let's rename the columns in our DataFrame to match the schema.

In [7]:
# Create a mapping from CSV columns to database columns
column_mapping = {
    'Id': 'Id',
    'START': 'START',
    'STOP': 'STOP',
    'PATIENT': 'PATIENT',
    'ORGANIZATION': 'ORGANIZATION',
    'PROVIDER': 'PROVIDER',
    'PAYER': 'PAYER',
    'ENCOUNTERCLASS': 'ENCOUNTERCLASS',
    'CODE': 'CODE',
    'DESCRIPTION': 'DESCRIPTION',
    'BASE_ENCOUNTER_COST': 'COST', # This is the key change
    'REASONCODE': 'REASONCODE',
    'REASONDESCRIPTION': 'REASONDESCRIPTION'
}

# Select and rename the columns
encounters_to_load = encounters_df[list(column_mapping.keys())].rename(columns=column_mapping)

print("DataFrame columns after renaming:")
print(encounters_to_load.columns.tolist())
encounters_to_load.head()

DataFrame columns after renaming:
['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'COST', 'REASONCODE', 'REASONDESCRIPTION']


,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION
0,748f8357-6cc7-551d-f31a-32fa2cf84126,2019-02-17T05:07:38Z,2019-02-17T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,NaN,NaN
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,2019-03-24T05:07:38Z,2019-03-24T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,NaN,NaN
2,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,2019-05-26T05:07:38Z,2019-05-26T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,NaN,NaN
3,6e93bcf9-45a4-8528-0120-1c1eaa930faf,2019-07-28T05:07:38Z,2019-07-28T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,NaN,NaN
4,8b6787c3-4316-a0cb-899d-4746525c319f,2019-10-27T05:07:38Z,2019-10-27T05:22:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,f7ae497d-8dc6-3721-9402-43b621a4e7d2,82608ebb-037c-3cef-9d34-3736d69b29e8,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,NaN,NaN


### Load the Transformed Data into PostgreSQL

In [8]:
try:
    encounters_to_load.to_sql('encounters', engine, if_exists='append', index=False)
    print("Successfully loaded data into 'encounters' table.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded data into 'encounters' table.


In [3]:
# Create a new cell and paste this code
patients_df = pd.read_csv('../synthea_sample_data_csv_nov2021/csv/patients.csv')

# The patients table has many columns. We will select and load all of them as they match the schema.
# No renaming is needed as the CSV headers match the table schema.
patients_to_load = patients_df

print("Columns from patients.csv:")
print(patients_to_load.columns.tolist())

try:
    patients_to_load.to_sql('patients', engine, if_exists='append', index=False)
    print("Successfully loaded data into 'patients' table.")
except Exception as e:
    print(f"An error occurred: {e}")

Columns from patients.csv:
['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']
Successfully loaded data into 'patients' table.


In [4]:
# Create a new cell and paste this code
conditions_df = pd.read_csv('../synthea_sample_data_csv_nov2021/csv/conditions.csv')

# The conditions table columns match the CSV headers directly.
conditions_to_load = conditions_df

print("Columns from conditions.csv:")
print(conditions_to_load.columns.tolist())

try:
    conditions_to_load.to_sql('conditions', engine, if_exists='append', index=False)
    print("Successfully loaded data into 'conditions' table.")
except Exception as e:
    print(f"An error occurred: {e}")

Columns from conditions.csv:
['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION']
Successfully loaded data into 'conditions' table.


In [5]:
# Create a new cell and paste this code
medications_df = pd.read_csv('../synthea_sample_data_csv_nov2021/csv/medications.csv')

# Here, we need to select the correct columns as the CSV has extra ones.
# The schema for 'medications' is: START, STOP, PATIENT, PAYER, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE, REASONDESCRIPTION
# The CSV has all of these, so no renaming is needed, but we ensure we only select these.

columns_to_select = [
    'START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 
    'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION'
]

medications_to_load = medications_df[columns_to_select]

print("Columns from medications.csv:")
print(medications_to_load.columns.tolist())

try:
    medications_to_load.to_sql('medications', engine, if_exists='append', index=False)
    print("Successfully loaded data into 'medications' table.")
except Exception as e:
    print(f"An error occurred: {e}")


Columns from medications.csv:
['START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION']
Successfully loaded data into 'medications' table.


In [7]:
# Create a new cell and paste this code
organizations_df = pd.read_csv('../synthea_sample_data_csv_nov2021/csv/organizations.csv')

# The organizations schema has a 'REVENUE' column, which is named 'REVENUE' in the CSV.
# The schema also has 'UTILIZATION', which is not in the organizations.csv, so it will be null.
# Let's ensure all columns match.
# DB Schema: Id, NAME, ADDRESS, CITY, STATE, ZIP, LAT, LON, PHONE, REVENUE, UTILIZATION
# CSV Columns: Id, NAME, ADDRESS, CITY, STATE, ZIP, LAT, LON, PHONE, REVENUE

columns_to_select = [
    'Id', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 
    'LAT', 'LON', 'PHONE', 'REVENUE'
]

# The 'UTILIZATION' column is in the DB but not the CSV. Pandas will handle this gracefully
# by inserting NULL for that column, as long as the DB allows NULLs.
organizations_to_load = organizations_df[columns_to_select]

print("Columns from organizations.csv:")
print(organizations_to_load.columns.tolist())

try:
    organizations_to_load.to_sql('organizations', engine, if_exists='append', index=False)
    print("Successfully loaded data into 'organizations' table.")
except Exception as e:
    print(f"An error occurred: {e}")


Columns from organizations.csv:
['Id', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LAT', 'LON', 'PHONE', 'REVENUE']
Successfully loaded data into 'organizations' table.
